This code block is to simplify my implementation. It is not important for this assignment, so please skip looking at it.

In [ ]:
from collections import Hashable, MutableSet, MutableSequence, MutableMapping

def make_hashdict(value):
    """
    Inspired by https://stackoverflow.com/questions/1151658/python-hashable-dicts
     - with the added bonus that it inherits from the dict type of value
       so OrderedDict's maintain their order and other subclasses of dict() maintain their attributes
    """
    map_type = type(value)

    class HashableDict(map_type):
        def __init__(self, *args, **kwargs):
            super(HashableDict, self).__init__(*args, **kwargs)
        def __hash__(self):
            return hash(tuple(sorted(self.items())))

    hashDict = HashableDict(value)

    return hashDict

def make_hashable(value):
    if not isinstance(value, Hashable):
        if isinstance(value, MutableSet):
            value = frozenset(value)
        elif isinstance(value, MutableSequence):
            value = tuple(value)
        elif isinstance(value, MutableMapping):
            value = make_hashdict(value)

        return value



The function "isCover" is a function to check if a set of sensors is a target coverage. It will receive three inputs where "coveredTarget[sensor]" is a set of target that "sensor" can monitor, "targetSet" is a set of all targets, and "sensorSet" is a a set of sensors which we want to check if it is target coverage.

An example of how to define "coveredTarget" and "targetSet" can be found at the end of the next code block.

In [ ]:
def isCover(coveredTarget, targetSet, sensorSet):
  coveredSet = set()
  for sensor in sensorSet:
    coveredSet = coveredSet | coveredTarget[sensor]
  if coveredSet == targetSet:
    return True
  else:
    return False

The function "getTargetCoverages" will return all target coverages from a given sensor network. It takes two inputs - "coveredTarget[sensor]" is the set of targets that "sensor" can monitor, and "targetSet" is the set of all targets.

An example of how to define "coveredTarget" and "targetSet" can be found at the end of the code block.

In [ ]:
from functools import reduce

def powerset(lst):
  return reduce(lambda result, x: result + [subset + [x] for subset in result], lst, [[]])

def getTargetCoverages(coveredTarget, targetSet):
  ps = powerset(range(len(coveredTarget)))
  targetCover = []
  for sensorSet in ps:
    if isCover(coveredTarget, targetSet, sensorSet):
      targetCover.append(sensorSet)
  return targetCover

allCoverages = getTargetCoverages([set([1,2]),set([2,3]),set([1,3])], set([1,2,3]))


The function "getBestTargetCoverage" will return the target coverage which have smallest cost. The function will take only one input "weightList[sensor]" which is going to be a cost of "sensor". We aim to find a target coverage $C$ such that $\sum_{sensor \in C} weightList[sensor]$ is minimized here.

An example how to call this function can be found at the end of this code block.

In [ ]:
def getBestTargetCoverage(weightList, allCoverages):
  toReturn = []
  minWeight = 10000
  for coverage in allCoverages:
    sumWeight = 0
    for sensor in coverage:
      sumWeight = sumWeight + weightList[sensor]
    if sumWeight < minWeight:
      minWeight = sumWeight
      toReturn = coverage
  return toReturn, minWeight

getBestTargetCoverage([0.007982945486089197, 0.007982945486089197, 0.007982945486089197], allCoverages)

([0, 1], 0.015965890972178393)

This code block is used for initialized all parameters.

In [ ]:
import math

#numSensor = 3
numSensor = 14
theta = 0.01
delta = (1 + theta)*pow(numSensor * (1 + theta), -1/theta)
tau = math.log(1 + theta) / (math.log(1 + theta) - math.log(delta))
theta, delta, tau

(0.01, 2.945672288445908e-131, 3.310506185376382e-05)

This code block is an implementation of the Garg-Konemann Framework. It uses the function "getBestTargetCoverage" to obtain the most problematic constraint.

In [ ]:
#allCoverages = getTargetCoverages([set([1,2]),set([2,3]),set([1,3])], set([1,2,3]))
allCoverages = getTargetCoverages([set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9]),set([6,7,10,11]),set([8,9,12,13]),set([10,11,14,15]),set([12,13,16,17]),set([14,15,18,19]), set([16, 17, 18, 19]), set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9])], set([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]))
print(len(allCoverages))
weightList = [delta] * numSensor
schedule = {}
while True:
  (targetCoverage, minWeight) = getBestTargetCoverage(weightList, allCoverages)
  if minWeight >= 1:
    break
  if make_hashable(targetCoverage) not in schedule.keys():
    schedule[make_hashable(targetCoverage)] = 0.0
  schedule[make_hashable(targetCoverage)] = schedule[make_hashable(targetCoverage)] + tau
  #print(targetCoverage, weightList, schedule)
  for sensor in targetCoverage:
    weightList[sensor] = weightList[sensor] * (1 + theta)
schedule, weightList
print("--- %s seconds ---" % (time.time() - start_time))  


3843


({(0, 3, 4, 7, 8): 0.8358697067454149,
  (0, 4, 7, 8, 13): 0.021650710452361475,
  (1, 2, 5, 6, 9): 0.8358697067454149,
  (1, 5, 6, 9, 12): 0.021650710452361475,
  (2, 5, 6, 9, 11): 0.03591899211133364,
  (3, 4, 7, 8, 10): 0.03591899211133364,
  (4, 7, 8, 10, 13): 0.10289053224149765,
  (5, 6, 9, 11, 12): 0.10289053224149765},
 [2.545244577686729e-19,
  2.545244577686729e-19,
  1.8545631290278968e-17,
  1.8545631290278968e-17,
  0.33515815539209026,
  0.33515815539209026,
  0.33515815539209026,
  0.33515815539209026,
  0.33515815539209026,
  0.33515815539209026,
  3.878865313141803e-113,
  3.878865313141803e-113,
  5.323442891386635e-115,
  5.323442891386635e-115,
  2.945672288445908e-131,
  2.945672288445908e-131,
  2.945672288445908e-131,
  2.945672288445908e-131,
  2.945672288445908e-131,
  2.945672288445908e-131])

In [ ]:
#returns how many new targets would be added by a sensor
def f_delta_targets_monitored(coverage, coveredTarget, sensor):
  monitored = set()
  count = 0
  for i in coverage:
    monitored = monitored.union(coveredTarget[i])
  for i in sensor:
    if i not in monitored:
      count+=1
  return count

#form coverage from input: targets[sensor] set coveredTarget, 
#all targets targetSet, weightList[sensor] as cost
def getApproximatelyBestTargetCoverage(coveredTarget, targetSet, weightList):
  return_set = []   #start with empty set S, treated as a list
  minWeight = 0
  while isCover(coveredTarget, targetSet, return_set)!= True: #same as while f(S) < f(V)
      max = 0
      for sensor in coveredTarget: #sensor is a set
        #print("sensor: ", sensor, coveredTarget.index(sensor), "weight: ", weightList[coveredTarget.index(sensor)])
        potential_value = f_delta_targets_monitored(return_set, coveredTarget, sensor) / weightList[coveredTarget.index(sensor)]
       #print("f_delta_targets_monitored(): ", f_delta_targets_monitored(return_set, coveredTarget, sensor))
       #print("potential_value: ", potential_value)
        if potential_value > max:
         # print("is max")
          max = potential_value
          max_sensor = sensor
      max_sensor_index = coveredTarget.index(max_sensor) #index in the coveredTarget list
      return_set.append(max_sensor_index)
      minWeight += weightList[max_sensor_index]
      #print("MAX SENSOR: ", max_sensor)
      #print("return set: ", return_set)
      #print("weight: ", minWeight)
  #print("RETURNED")
  return return_set, minWeight


In [ ]:
import time
start_time = time.time()
#coveredTarget = [set([1,2]),set([2,3]),set([1,3])] 
#targetSet = set([1,2,3])#number of total targets
coveredTarget = [set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9]),set([6,7,10,11]), set([8,9,12,13]), set([10,11,14,15]),set([12,13,16,17]),set([14,15,18,19]), set([16, 17, 18, 19]), set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9])]
targetSet = set([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

#allCoverages = getTargetCoverages([set([1,2]),set([2,3]),set([1,3])], set([1,2,3]))
#allCoverages = getTargetCoverages([set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9]),set([6,7,10,11]),set([8,9,12,13]),set([10,11,14,15]),set([12,13,16,17]),set([14,15,18,19]), set([16, 17, 18, 19]), set([0,1,2,3]),set([0,1,4,5]),set([2,3,6,7]), set([4,5,8,9])], set([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]))
#print(len(allCoverages))
weightList = [delta] * numSensor

schedule = {}
while True:
  (targetCoverage, minWeight) = getApproximatelyBestTargetCoverage(coveredTarget, targetSet, weightList)
  #(targetCoverage, minWeight) = getBestTargetCoverage(weightList, allCoverages)
  if minWeight >= 1:
    break
  if make_hashable(targetCoverage) not in schedule.keys():
    schedule[make_hashable(targetCoverage)] = 0.0
  schedule[make_hashable(targetCoverage)] = schedule[make_hashable(targetCoverage)] + tau
  #print(targetCoverage, weightList, schedule)
  for sensor in targetCoverage:
    weightList[sensor] = weightList[sensor] * (1 + theta)
print(schedule, weightList)
print("--- %s seconds ---" % (time.time() - start_time))


{(0, 3, 4, 7, 8): 0.9946415833958661, (1, 2, 5, 6, 9): 0.9946415833958661} [0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 0.20177145632626023, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131, 2.945672288445908e-131]
--- 6.224792718887329 seconds ---
